# Model Deployment

## 1. Loading the saved Model

In [ ]:
!pip install wandb

In [ ]:
import pandas as pd
from joblib import load
import warnings
warnings.filterwarnings("ignore")


import os
import wandb
import sklearn
from joblib import load
print(sklearn.__version__)

1.5.2


In [ ]:
class AirlineSatisfactionPredictionModel():

    def __init__(self, model, features, rmse):
        self.model = model
        self.features = features
        self.rmse = rmse

In [ ]:
os.environ["WANDB_API_KEY"] = "2dd7cb2aa4dead52f2776ea6e5f391577474bbdb"

In [ ]:
run = wandb.init(project="mlops_airlinesatisfaction", job_type="inference")
artifact = run.use_artifact("LogisticRegression_AirlineSatisfaction:latest")

model_dir = artifact.download()
model_path = os.path.join(model_dir, 'airline - logreg.pkl')
model_v1 = load(model_path)
#model_path = artifact.file()

wandb:   2 of 2 files downloaded.  


In [ ]:
# Load the scikit-learn model
model_v1 = load(model_path)

print(model_v1)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num_not_imputed',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Arrival Delay in Minutes',
                                                   'Flight Distance',
                                                   'Departure Delay in Minutes',
                                                   'Age']),
                                                 ('catvars',
                                                  Pipeline(steps=[('ohencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Gender', 'Customer Type',
                                                   'Type of Travel', 'Class',
                                    

## Preparing the record for New Airline Customer

In [ ]:
airline_dict = {
    "Gender": "F", # This is the gender, and takes male or female
    "Customer Type": "loyal customer", # This takes loyal customer/disloyal customer
    "Age": 40, # This takes age values ranging from 7 to 85
    "Type of Travel": "Business Travel", # Personal Travel/Business Travel
    "Class": "Business", # Eco/Eco Plus/Business
    "Flight Distance": 3000, # Distance of the flight max 4963
    "Inflight wifi service": 4, # Rating of the service 0 to 5
    "Departure/Arrival time convenient": 3, # Rating of the service 0 to 5
    "Ease of Online booking": 5, # Rating of the service 0 to 5
    "Gate location": 4, # Rating of the service 0 to 5
    "Food and drink": 4, # Rating of the service 0 to 5
    "Online boarding": 5,  # Rating of the service 0 to 5
    "Seat comfort": 5, # Rating of the service 0 to 5
    "Inflight entertainment": 4, # Rating of the service 0 to 5
    "On-board service": 5, # Rating of the service 0 to 5
    "Leg room service": 5, # Rating of the service 0 to 5
    "Baggage handling": 4, # Rating of the service 0 to 5
    "Checkin service": 5, # Rating of the service 0 to 5
    "Inflight service": 4, # Rating of the service 0 to 5
    "Cleanliness": 3, # Rating of the service 0 to 5
    "Departure Delay in Minutes": 30, # Duration of delay in departure
    "Arrival Delay in Minutes": 33, # Duration of delay in arrival
}

In [ ]:
df = pd.DataFrame(airline_dict, index = [0])

In [ ]:
df

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,F,loyal customer,40,Business Travel,Business,3000,4,3,5,4,...,5,4,5,5,4,5,4,3,30,33


## Predicting the Customer Experience

In [ ]:
prediction = model_v1.predict(df)

prediction

array([1])

In [ ]:
if prediction==0:
           print('Expected Customer Satisfaction: Neutral or Dissatisfied')
else:
            print('Expected Customer Satisfaction: Satisfied')

Expected Customer Satisfaction: Satisfied


## Model Deployment using Ngrok

In [ ]:
!pip install fastapi uvicorn wandb pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
!ngrok authtoken 2oyADN6RZHswS0f2EJZvYmqkEDG_7dmNuEzguhDsRpM78y2Bi

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
from google.colab import userdata
os.environ["WANDB_API_KEY"] = "2dd7cb2aa4dead52f2776ea6e5f391577474bbdb"

In [ ]:
%%writefile app.py

import os
import wandb
import sklearn
from joblib import load
from fastapi import FastAPI, HTTPException
from contextlib import asynccontextmanager
from pydantic import BaseModel
import wandb
import joblib
import pandas as pd
import numpy as np

app = FastAPI()

# Define input data schema
class PredictionRequest(BaseModel):
    Gender: str
    Customer_Type: str
    Age: int
    Type_of_Travel: str
    Class: str
    Flight_Distance: int
    Inflight_wifi_service: int
    Departure_Arrival_time_convenient: int
    Ease_of_Online_booking: int
    Gate_location: int
    Food_and_drink: int
    Online_boarding: int
    Seat_comfort: int
    Inflight_entertainment: int
    On_board_service: int
    Leg_room_service: int
    Baggage_handling: int
    Checkin_service: int
    Inflight_service: int
    Cleanliness: int
    Departure_Delay_in_Minutes: int
    Arrival_Delay_in_Minutes: int

# Load the model from WandB
def load_model_from_wandb(project_name: str, artifact_name: str):
    try:
        # Initialize WandB
        run = wandb.init(project=project_name, job_type="inference", reinit=True)
        artifact = run.use_artifact(artifact_name)
        model_path = artifact.file()  # Assumes a single model file in the artifact
        model = joblib.load(model_path)
        run.finish()  # End the WandB run
        return model
    except Exception as e:
        raise RuntimeError(f"Failed to load model: {e}")

project_name = "mlops_AirlineSatisfaction"  # Replace with your project name
artifact_name = "Linear_Model_AirlineSatisfaction:latest"  # Replace with your artifact name
ml_model = load_model_from_wandb(project_name, artifact_name)

# Prediction endpoint
@app.post("/predict")
def predict(input_data: PredictionRequest):
    try:
        # Convert input data to a dictionary for prediction
        input_dict = input_data.dict()

        df = pd.DataFrame(input_dict, index = [0])

        # Call the model's prediction method
        prediction = ml_model.predict(df)

        # Return the prediction result
        return {f"Estimated Car Price: INR {np.round(prediction[0], 2)} Lakhs"}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {e}")


Writing app.py


In [ ]:
!nohup uvicorn app:app --host 0.0.0.0 --port 8000 &

nohup: appending output to 'nohup.out'


In [ ]:
!cat nohup.out

In [ ]:
!ps -ax | grep uvicorn

   2667 ?        Sl     0:04 /usr/bin/python3 /usr/local/bin/uvicorn app:app --host 0.0.0.0 --port 8
   2714 ?        S      0:00 /bin/bash -c ps -ax | grep uvicorn
   2716 ?        S      0:00 grep uvicorn


In [ ]:
from pyngrok import ngrok

# Expose the FastAPI app
public_url = ngrok.connect(8000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://de04-146-148-75-177.ngrok-free.app" -> "http://localhost:8000"
